# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [8]:
import napari
from macrohet import dataio, tile
import btrack
print(btrack.__version__)
import os

0.6.1.dev31


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [10]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [4]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Define row and column of choice

In [11]:
row = 3
column = 5

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [6]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane=1,#'sum_proj',
#                              set_channel=1,
#                              set_time = 66,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()

In [ ]:
%%time
images = images.compute().compute()

# Load tracks

In [20]:
tracks_fn = os.path.join(base_dir, f'labels/full_localisation/{row,column}_objs_masks_tracks.h5')
with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
    tracks = hdf.tracks
    segmentation = hdf.segmentation
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)

[INFO][2023/04/26 03:08:45 PM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5...
[INFO][2023/04/26 03:08:45 PM] Loading tracks/obj_type_1
[INFO][2023/04/26 03:08:45 PM] Loading objects/obj_type_1 (31313, 5) (31313 filtered: None)
[INFO][2023/04/26 03:09:04 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/04/26 03:09:04 PM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5


### Load downscaled version

In [19]:
tracks_fn = os.path.join(base_dir, f'labels/full_localisation/downscaled/{row,column}_downsc_objs_masks_tracks.h5')
with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
    ds_tracks = hdf.tracks
    ds_segmentation = hdf.segmentation
ds_napari_tracks, properties, graph = btrack.utils.tracks_to_napari(ds_tracks, ndim=2)

[INFO][2023/04/28 11:47:20 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/(3, 5)_downsc_objs_masks_tracks.h5...
[INFO][2023/04/28 11:47:20 AM] Loading tracks/obj_type_1
[INFO][2023/04/28 11:47:20 AM] Loading LBEP/obj_type_1
[INFO][2023/04/28 11:47:20 AM] Loading objects/obj_type_1 (26917, 5) (26917 filtered: None)
[INFO][2023/04/28 11:47:21 AM] Loading segmentation (75, 1200, 1200)
[INFO][2023/04/28 11:47:21 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/(3, 5)_downsc_objs_masks_tracks.h5


### Load retracked version

In [26]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/relocalisation/relocalised_{row, column}.h5'), 
                                       'r', 
                                       obj_type='obj_type_1'
                                       ) as reader:
            tracks = reader.tracks
            objects = reader.objects
            segmentation = reader.segmentation
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)

[INFO][2023/04/28 11:51:10 am] Opening HDF file: /mnt/DATA/macrohet/labels/relocalisation/relocalised_(3, 5).h5...
28-Apr-23 11:51:10 - btrack.io.hdf - INFO     - Opening HDF file: /mnt/DATA/macrohet/labels/relocalisation/relocalised_(3, 5).h5...
[INFO][2023/04/28 11:51:10 am] Loading tracks/obj_type_1
28-Apr-23 11:51:10 - btrack.io.hdf - INFO     - Loading tracks/obj_type_1
[INFO][2023/04/28 11:51:10 am] Loading LBEP/obj_type_1
28-Apr-23 11:51:10 - btrack.io.hdf - INFO     - Loading LBEP/obj_type_1
[INFO][2023/04/28 11:51:11 am] Loading objects/obj_type_1 (31313, 5) (31313 filtered: None)
28-Apr-23 11:51:11 - btrack.io.hdf - INFO     - Loading objects/obj_type_1 (31313, 5) (31313 filtered: None)
[INFO][2023/04/28 11:51:11 am] Loading objects/obj_type_1 (31313, 5) (31313 filtered: None)
28-Apr-23 11:51:11 - btrack.io.hdf - INFO     - Loading objects/obj_type_1 (31313, 5) (31313 filtered: None)
[INFO][2023/04/28 11:51:39 am] Loading segmentation (75, 6048, 6048)
28-Apr-23 11:51:39 - btr

### Recolour tracks

In [21]:
ds_col_segmentation = btrack.utils.update_segmentation(ds_segmentation, ds_tracks,)

In [27]:
ds_col_segmentation_new_tracks = btrack.utils.update_segmentation(ds_segmentation, tracks,)

In [16]:
col_segmentation = btrack.utils.update_segmentation(segmentation, tracks)

# Launch napari image viewer

In [28]:
viewer = napari.Viewer()

viewer.add_image(images, 
                 channel_axis=1,
                 name=["macrophage", "mtb"],
                 colormap=["green",  "magenta"],
#                  contrast_limits=[[100, 6000], [100, 2000]],
                 contrast_limits=[[0,450], [0,450]], 
                 visible = False
                 )
# viewer.add_labels(segmentation, 
#                   num_colors= 1,
#                   #scale=(10, 1, 1,), 
#                   #color='yellow'
#                   name = 'original segmentation'
#                  )
# viewer.add_labels(col_segmentation, 
#                   name = 'recolored original segmentation'
#                   #scale=(10, 1, 1,), 
#                   #color='yellow'
#                  )
viewer.add_tracks(napari_tracks,
                    properties=properties, 
                    graph=graph, 
                    name="Properly downscaled tracks", 
                    blending="translucent",
                    visible=True,
#                     scale = (100,1,1)
                )
viewer.add_tracks(ds_napari_tracks,
#                     properties=properties, 
#                     graph=graph, 
                    name="hacky downscaled tracks", 
                    blending="translucent",
                    visible=True,
#                     scale = (1,1,1)
                )
# viewer.add_labels(ds_segmentation, 
#                  name = 'downscaled segmentation')

viewer.add_labels(ds_col_segmentation, 
                  name = 'downscaled coloured segmentation',
                  scale=(1, 1, 1,), 
#                   color={1:'yellow'}
                 )

viewer.add_labels(ds_col_segmentation_new_tracks, 
                  name = 'downscaled coloured segmentation new tracks',
                  scale=(1, 1, 1,), 
#                   color={1:'yellow'}
                 )

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'downscaled coloured segmentation [1]' at 0x7f15c2c4f4f0>

In [32]:
import numpy as np

In [33]:
viewer.add_image(np.zeros((10,10)))

<Image layer 'Image' at 0x7f15c2638ac0>

In [29]:
viewer.add_labels(ds_col_segmentation_new_tracks, 
                  name = 'downscaled coloured segmentation new tracks',
                  scale=(1, 1, 1,), 
#                   color={1:'yellow'}
                 )

<Labels layer 'downscaled coloured segmentation new tracks' at 0x7f15c273ca30>

In [36]:
[track for track in ds_tracks if track.ID == 395][0]

,ID,t,x,y,z,parent,root,state,generation,dummy,mean_intensity-1,mean_intensity-0,area,major_axis_length,minor_axis_length,orientation
0,395,0,202.988235,564.007874,0.000000,395,395,5,0,False,0.005599,0.005862,510.0,28.871780,22.532621,0.453096
1,395,1,202.988235,564.007874,0.000000,395,395,5,0,True,NaN,NaN,NaN,NaN,NaN,NaN
2,395,2,211.879364,567.457153,0.000000,395,395,5,0,False,0.005646,0.007447,630.0,32.615505,25.099262,0.626124
3,395,3,214.965103,565.377869,0.000000,395,395,5,0,False,0.005631,0.006578,659.0,35.717762,23.640097,0.120743
4,395,4,215.229996,567.748596,0.000000,395,395,5,0,False,0.005659,0.010566,700.0,40.367668,22.323833,0.113697
5,395,5,215.954254,565.537231,0.000000,395,395,5,0,False,0.005640,0.007536,765.0,40.883957,24.781647,0.481782
6,395,6,218.932159,573.566345,0.000000,395,395,5,0,False,0.005659,0.008833,1017.0,46.987591,28.146175,0.346010
7,395,7,217.275085,572.713623,0.000000,395,395,5,0,False,0.005633,0.006755,887.0,47.992393,24.330168,0.190410
8,395,8,217.589005,574.258240,0.000000,395,395,5,0,False,0.005662,0.008764,910.0,47.003250,25.064434,0.094458
9,395,9,220.816376,573.343689,0.000000,395,395,5,0,False,0.005665,0.009218,806.0,44.436798,23.392088,0.176147


In [25]:
viewer = napari.Viewer()

viewer.add_image(images, 
                 channel_axis=1,
                 name=["macrophage", "mtb"],
                 colormap=["green",  "magenta"],
#                  contrast_limits=[[100, 6000], [100, 2000]],
                 contrast_limits=[[0,450], [0,450]], 
                 visible = False
                 )
viewer.add_labels(segmentation, 
                  num_colors= 1,
                  #scale=(10, 1, 1,), 
                  #color='yellow'
                  name = 'original segmentation'
                 )
viewer.add_labels(col_segmentation, 
                  name = 'recolored original segmentation'
                  #scale=(10, 1, 1,), 
                  #color='yellow'
                 )
viewer.add_tracks(napari_tracks,
                    properties=properties, 
                    graph=graph, 
                    name="Properly downscaled tracks", 
                    blending="translucent",
                    visible=True,
#                     scale = (100,1,1)
                )
viewer.add_tracks(ds_napari_tracks,
#                     properties=properties, 
#                     graph=graph, 
                    name="hacky downscaled tracks", 
                    blending="translucent",
                    visible=True,
#                     scale = (1,1,1)
                )
viewer.add_labels(ds_segmentation, 
                 name = 'downscaled segmentation')

viewer.add_labels(ds_col_segmentation, 
                  name = 'downscaled coloured segmentation',
                  scale=(1, 1, 1,), 
#                   color={1:'yellow'}
                 )

/home/dayn/analysis/btrack/btrack/dataio.py:3: UserWarning: `btrack.dataio` has been deprecated. Please use `btrack.io` subpackage instead.
  warnings.warn(  # noqa: B028


Assistant skips harvesting pyclesperanto as it's not installed.


/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'downscaled coloured segmentation' at 0x7f14ffc7a7c0>

In [55]:
color_dict = dict([(i, 'cyan') for i in range(1, segmentation[0].max())])
viewer = napari.Viewer()

viewer.add_labels(segmentation[0], num_colors = 1, color = color_dict).contour = 10
                  #scale=(10, 1, 1,), 
                  #color='yellow'
#                  )

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [56]:
viewer.add_image(images[0], 
                 channel_axis=0,
                 name=["macrophage", "mtb"],
                 colormap=["green",  "magenta"],
                 contrast_limits=[[0,450], [0,450]], 
                 visible = False
                 )

[<Image layer 'macrophage' at 0x7f71498ef760>,
 <Image layer 'mtb' at 0x7f714720e400>]

Rendering frames...


100%|█████████████████████████████████████████████████████████████████| 401/401 [1:11:16<00:00, 10.66s/it]


In [71]:
viewer = napari.Viewer()

viewer.add_image(images, 
                 channel_axis=1,
                 name=["macrophage", "mtb"],
                 colormap=["green",  "magenta"],
#                  contrast_limits=[[100, 6000], [100, 2000]],
                 contrast_limits=[[0,450], [0,450]], 
                 visible = False
                 )
viewer.add_labels(segmentation, 
                  #scale=(10, 1, 1,), 
                  #color='yellow'
                 )
# viewer.add_tracks(napari_tracks,
# #                     properties=properties, 
# #                     graph=graph, 
# #                     name="Tracks [5,4]", 
#                     blending="translucent",
#                     visible=True,
# #                     scale = (100,1,1)
# #                 )
viewer.add_tracks(ds_napari_tracks,
#                     properties=properties, 
#                     graph=graph, 
#                     name="Tracks [5,4]", 
                    blending="translucent",
                    visible=True,
                    scale = (1,1,1)
                )

viewer.add_labels(ds_col_segmentation, 
                  scale=(1, 1, 1,), 
#                   color={1:'yellow'}
                 )

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'ds_col_segmentation' at 0x7f713d007b80>

In [73]:
viewer = napari.Viewer()

viewer.add_image(images, 
                 channel_axis=1,
                 name=["macrophage", "mtb"],
                 colormap=["green",  "magenta"],
#                  contrast_limits=[[100, 6000], [100, 2000]],
                 contrast_limits=[[0,450], [0,450]], 
                 visible = False
                 )
viewer.add_labels(segmentation, 
                  #scale=(10, 1, 1,), 
                  #color='yellow'
                 )
# viewer.add_tracks(napari_tracks,
# #                     properties=properties, 
# #                     graph=graph, 
# #                     name="Tracks [5,4]", 
#                     blending="translucent",
#                     visible=True,
# #                     scale = (100,1,1)
# #                 )
viewer.add_tracks(ds_napari_tracks,
#                     properties=properties, 
#                     graph=graph, 
#                     name="Tracks [5,4]", 
                    blending="translucent",
                    visible=True,
                    scale = (1,1,1)
                )

viewer.add_labels(ds_col_segmentation, 
                  scale=(1, 1, 1,), 
#                   color={1:'yellow'}
                 )

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'ds_col_segmentation' at 0x7f713fdd1250>

In [78]:
tracker_layer = viewer.add_tracks(napari_tracks_us,
#                     properties=properties, 
#                     graph=graph, 
#                     name="Tracks [5,4]", 
                    blending="translucent",
                    visible=True,
                    scale = (1,1,1)
                )

In [63]:
from macrohet import visualise

In [76]:
napari_tracks_us = visualise.scale_napari_tracks(ds_napari_tracks)

In [ ]:
@viewer.bind_key('p')
def Mtb_pos(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "SPC_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "SPC_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@viewer.bind_key('n')
def Mtb_neg(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "SPC_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "SPC_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")

# Misc:

Downscale images, binarise masks, filter tracks

In [42]:
for i in [20, 40, 60, 80]:
    viewer.add_tracks(btrack.utils.tracks_to_napari([track for track in ds_tracks if len(track) < i],ndim=2)[0], 
                      name = i,
#                     properties=properties, 
#                     graph=graph, 
#                     name="Tracks [5,4]", 
                    blending="translucent",
                    visible=True,
                    scale = (10,1,1)
                )

In [24]:
import numpy as np
from skimage.morphology import square, binary_erosion, remove_small_objects
from tqdm.auto import tqdm

In [58]:
viewer.add_tracks(ds_napari_tracks,
#                     properties=properties, 
#                     graph=graph, 
#                     name="Tracks [5,4]", 
                    blending="translucent",
                    visible=True,
                    scale = (10,1,1)
                )

NameError: name 'ds_napari_tracks' is not defined

In [51]:
binary_masks = np.zeros(masks.shape, dtype = np.uint8)
for n, mask in tqdm(enumerate(masks), 
                 total = len(masks), 
                 desc = 'Progress through mask stack'):
#     binary_mask = np.zeros(mask.shape, dtype = np.uint8)
    binary_mask = binary_erosion(mask, square(2))
#     mask = remove_small_objects(mask, min_size=500)
#     for segment_ID in tqdm(range(1, np.max(mask)), 
#                            total = np.max(mask), 
#                            desc = 'Progress through number of segments', 
#                            leave = False):
#         segment = mask == segment_ID 
#         eroded_segment = binary_erosion(segment, square(5))
#         binary_mask += eroded_segment.astype(np.uint8)
    binary_masks[n] = binary_mask.astype(np.uint8) 

Progress through mask stack:   0%|          | 0/75 [00:00<?, ?it/s]

##### Downscale final image

In [61]:
from skimage.transform import rescale, resize, downscale_local_mean

In [60]:
final_frame = images[-1].compute().compute()

In [62]:
gfp = final_frame[0]
rfp = final_frame[1]
gfp_images_resized = list()
for t in tqdm(range(len(gfp)), 
              desc = f'Resizing GFP images in position {row, column}', leave = False):
    gfp_image_resized = rescale(gfp[t], 1200/6048, anti_aliasing=False)
    gfp_images_resized.append(gfp_image_resized)
gfp_images_resized = np.stack(gfp_images_resized, axis = 0)
rfp_images_resized = list()
for t in tqdm(range(len(rfp)),  
              desc = f'Resizing RFP images in position {row, column}', leave = False):
    rfp_image_resized = rescale(rfp[t], 1200/6048, anti_aliasing=False)
    rfp_images_resized.append(rfp_image_resized)
rfp_images_resized = np.stack(rfp_images_resized, axis = 0)
### stack together resized images for localisation
resized_images = np.stack([gfp_images_resized,rfp_images_resized], axis = -1)

Resizing GFP images in position ('5', '4'):   0%|          | 0/1 [00:00<?, ?it/s]

Resizing RFP images in position ('5', '4'):   0%|          | 0/1 [00:00<?, ?it/s]

In [64]:
resized_images.shape

(1, 1200, 1200, 2)